# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd

In [2]:
bank_data = pd.read_csv('/datasets/data.csv')

In [3]:
bank_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Описание данных:
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

**Вывод**

В каждой строке таблицы — статистика о платёжеспособности клиентов. Колонки рассказывают о клиентах банка.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
# подсчёт пропусков
bank_data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
bank_data[bank_data['days_employed'].isna() & bank_data['total_income'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Пропуски значений в столбцах `days_employed` и `total_income` неслучайные.

Пропуски количественых значений в столбцах `days_employed` и `total_income` заменим на среднее арифметическое или медианное значение.

In [7]:
# получение общей информации о столбце с общим трудовым стажом клиента в днях
bank_data.days_employed.describe().to_frame()

,days_employed
count,19351.000000
mean,63046.497661
std,140827.311974
min,-18388.949901
25%,-2747.423625
50%,-1203.369529
75%,-291.095954
max,401755.400475


Так как количество дней в трудовом стаже не может быть отрицательным заменим их на положительные значения.

In [8]:
bank_data.days_employed = bank_data.days_employed.apply(lambda x: x*(-1) if x < 0 else x)

In [9]:
# подсчет количества лет трудового стажа у клиента с максимальным количеством дней стажа
bank_data.days_employed.max() / 365.25

1099.9463394259549

В среднем живой человек не может отработать 1099 лет за свою жизнь, так как работать можно с 16 лет, будем считать некорректными значения, который больше возвраста клиента минус 16 лет. Прировняем такие значения к максимально возможному трудовому стажу: возраст клиента минус 16 лет.

In [10]:
# прировняем значение, которые больше возвраста клиента минус 16 лет, к максимально возможному трудовому стажу клиента
bank_data.loc[bank_data['days_employed'] / 365.25 > bank_data['dob_years'] - 16, 'days_employed'] = \
bank_data.loc[bank_data['days_employed'] / 365.25 > bank_data['dob_years'] - 16, 'dob_years'] * 365.25

In [11]:
# отношение среднего арифметического к медиане столбца с общим трудовым стажом клиента в днях
bank_data.days_employed.mean()/bank_data.days_employed.median()

2.657939078537321

Среднее арифметическое значения в 2.65 раза больше медианного значения трудового стажа клиента, значит некоторые значения сильно выделяются среди большинства. Для замены пропусков в столбце `days_employed` будем использовать медианное значение.

In [12]:
# медианное значение трудового стажа
median_days_employed = bank_data.days_employed.median()
# заменяем пропуски в столбце total_income медианным значением трудового стажа всех клиентов
bank_data.days_employed.fillna(median_days_employed, inplace=True)

In [13]:
# подсчёт пропусков
bank_data.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [14]:
# получение общей информации о столбце с ежемесячным доходом клиентов
bank_data.total_income.describe().to_frame()

,total_income
count,1.935100e+04
mean,1.674223e+05
std,1.029716e+05
min,2.066726e+04
25%,1.030532e+05
50%,1.450179e+05
75%,2.034351e+05
max,2.265604e+06


In [15]:
bank_data.total_income.mean()/bank_data.total_income.median()

1.1544937478551978

Среднее арифметическое значения в 1.15 раза больше медианного значения трудового стажа клиента, значит некоторые значения не сильно выделяются среди большинства. Для замены пропусков в столбце `total_income` будем использовать среднее арифметическое значение.

In [16]:
# Среднее арифметическое значение ежемесячного дохода
mean_total_income = bank_data.total_income.mean()
# заменяем пропуски в столбце total_income средним арифметическим значением ежемесячного дохода всех клиентов
bank_data.total_income.fillna(mean_total_income, inplace=True)

In [17]:
# подсчёт пропусков
bank_data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

 Обнаружены пропущеные количественные значения в столбцах `days_employed` и `total_income`. В столбце `days_employed` пропуски были заменины на медианное значение столбца, а в столбце `total_income` пропуски заменены на среднее арифмитическое заначение. Появления пропусков в данных возможны из-за технологической ошибки.

### Замена типа данных

In [18]:
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Заменим вещественный тип данных на целочисленный в столбцах `days_employed` и `total_income`

In [19]:
# округляю и перевожу в целлочисленный тип days_employed
bank_data.days_employed = bank_data.days_employed.round().astype('int')
# округляю и перевожу в целлочисленный тип total_income
bank_data.total_income = bank_data.total_income.round().astype('int')

In [20]:
# проверка результата замены типа данных
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Заменили вещественный тип данных на целочисленный в столбцах `days_employed` и `total_income` методом `astype('int')`, потому этот метод позваляет нам перевести значения в нужный тип.

### Обработка дубликатов

In [21]:
# подсчёт явных дубликатов
bank_data.duplicated().sum()

54

In [22]:
# вывод уникальных значений столбца education
bank_data.education.unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

В колонке `education` есть дубли написаные в разных регистрах, приведем их к одному регистру.

In [23]:
# приводим значение к нижнему регистру
bank_data.education = bank_data.education.str.lower()

In [24]:
# подсчёт явных дубликатов
bank_data.duplicated().sum()

71

Количество явных дубликатов увеличелось, теперь удалим их.

In [25]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
bank_data = bank_data.drop_duplicates().reset_index(drop=True)

In [26]:
# проверка на отсутствие дубликатов
bank_data.duplicated().sum()

0

**Вывод**

 Были найдены явные дубликаты методом `duplicated`, также после приведения значение столбца `education` к нижнему регистру количество явных дубликатов увеличилось. Дубликаты были удалены методом `drop_duplicates`. Появление дубликатов в данных возможны из-за технической ошибки при копировании и человеческого фактора при заполнении информации.

### Лемматизация

In [27]:
# импортируем библиотеку для лемматизации
from pymystem3 import Mystem
m = Mystem()

In [28]:
# создание нового столбца с пустыми значениями
bank_data['lemmas_purpose'] = ''
# заполнение значений столбца lemmas_purpose леммами  из строк столбца purpose
for i in range(len(bank_data['purpose'])):
    bank_data.at[i,'lemmas_purpose']  = m.lemmatize(bank_data['purpose'][i])

In [29]:
bank_data.lemmas_purpose.value_counts().reset_index()

,index,lemmas_purpose
0,"[автомобиль, \n]",972
1,"[свадьба, \n]",791
2,"[на, , проведение, , свадьба, \n]",768
3,"[сыграть, , свадьба, \n]",765
4,"[операция, , с, , недвижимость, \n]",675
5,"[покупка, , коммерческий, , недвижимость, \n]",661
6,"[операция, , с, , жилье, \n]",652
7,"[покупка, , жилье, , для, , сдача, \n]",651
8,"[операция, , с, , коммерческий, , недвижимо...",650
9,"[покупка, , жилье, \n]",646


**Вывод**

Благодаря леммонтизации столбца `purpose` цель получения кредита можно разделить на 4 категории: образование, автомобиль, свадьба, недвижимость.

### Категоризация данных

In [30]:
# создание функции для отбора цели получения кредита из лемм
def find_purpose(row):
    if  'образование' in row:
        return 'образование'
    elif 'автомобиль' in row:
        return 'автомобиль'
    elif 'свадьба' in row:
        return 'свадьба'
    else:
        return 'недвижимость'

In [31]:
# создание и заполнение нового столбца функцией find_purpose
bank_data['category_purpose'] = bank_data['lemmas_purpose'].apply(find_purpose)

In [32]:
bank_data['category_purpose'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: category_purpose, dtype: int64

In [33]:
# создание функции для группировки ежемесячного дохода по категориям
def total_income_group(income):
    if income < 100000:
        return 'низкий'
    elif 100000 < income < 200000:
        return 'средний'
    else:
        return 'высокий'

In [34]:
# создание и заполнение нового столбца функцией total_income_group
bank_data['total_income_group'] = bank_data['total_income'].apply(total_income_group)

In [35]:
bank_data['total_income_group'].value_counts()

средний    11925
высокий     5066
низкий      4463
Name: total_income_group, dtype: int64

**Вывод**

- Чаще всего цель получения кредита это недвижимость, а реже всего свадьба. 
- Разделяем клиентов по ежемесячному доходу на группы, больше всего клиентов со средним доходом.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [36]:
# проверка общей информации о количесте детей у клиетов
bank_data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Количество детей не может быть отрицательным, умножим отрицательные значения на -1

In [37]:
bank_data.children = bank_data.children.apply(lambda x: x*(-1) if x < 0 else x)

In [38]:
# проверка общей информации о количесте детей у клиетов
bank_data['children'].value_counts()

0     14091
1      4855
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [39]:
# создание новой колонкой со значениями True если есть ребенок
bank_data['have_children'] = bank_data['children'] > 0

In [40]:
# создание свободной таблици
have_children_pivot = bank_data.pivot_table(index=['have_children'], columns='debt', values='education_id', aggfunc='count')
# добавление колонки с конверсией вернувших в срок кредит
have_children_pivot['returned_on_time_share'] = have_children_pivot[1] / have_children_pivot.sum(numeric_only=True, axis=1)
have_children_pivot

debt,0,1,returned_on_time_share
have_children,,,
False,13028,1063,0.075438
True,6685,678,0.092082


**Вывод**

Клиенты с детьми чаще возвращают кредит в срок, чем клиенты без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [41]:
# проверка общей информации о семейном положение клиентов
bank_data.family_status.value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [42]:
# создание свободной таблици с семейным статусом
family_pivot = bank_data.pivot_table(index=['family_status'], columns='debt', values='education_id', aggfunc='count')
# добавление колонки с конверсией вернувших в срок кредит
family_pivot['returned_on_time_share'] = family_pivot[1]/family_pivot.sum(numeric_only=True, axis=1)
family_pivot.sort_values('returned_on_time_share')

debt,0,1,returned_on_time_share
family_status,,,
вдовец / вдова,896,63,0.065693
в разводе,1110,85,0.071130
женат / замужем,11408,931,0.075452
гражданский брак,3763,388,0.093471
Не женат / не замужем,2536,274,0.097509


**Вывод**

Не женатые / не замужем и находящиеся в гражданском браке клиенты чаще возращают кредит в срок, чем женатые / замужем и находящиеся в разводе клиенты, реже всего возвращают кредит в срок вдовец / вдова.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [43]:
# просмотр общей информации сгруппированном уровнем дохода клиентов
bank_data['total_income_group'].value_counts()

средний    11925
высокий     5066
низкий      4463
Name: total_income_group, dtype: int64

In [44]:
# создание свободной таблици с ежемесечным доходом
total_income_pivot = bank_data.pivot_table(index=['total_income_group'], columns='debt', values='education_id', aggfunc='count')
# добавление колонки с конверсией вернувших в срок кредит
total_income_pivot['returned_on_time_share'] = total_income_pivot[1]/total_income_pivot.sum(numeric_only=True, axis=1)
total_income_pivot.sort_values('returned_on_time_share')

debt,0,1,returned_on_time_share
total_income_group,,,
высокий,4708,358,0.070667
низкий,4109,354,0.079319
средний,10896,1029,0.086289


**Вывод**

Клиенты со средним доходом чаще остальных возвращают кредита в срок, а реже всех клиенты с высоким доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [45]:
# просмотр информации о цели кредита клиентов
bank_data['category_purpose'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: category_purpose, dtype: int64

In [46]:
# создание свободной таблици с целями получения кредита
purpose_pivot = bank_data.pivot_table(index=['category_purpose'], columns='debt', values='education_id', aggfunc='count')
# добавление колонки с конверсией вернувших в срок кредит
purpose_pivot['returned_on_time_share'] = purpose_pivot[1]/purpose_pivot.sum(numeric_only=True, axis=1)
purpose_pivot.sort_values('returned_on_time_share')

debt,0,1,returned_on_time_share
category_purpose,,,
недвижимость,10029,782,0.072334
свадьба,2138,186,0.080034
образование,3643,370,0.092200
автомобиль,3903,403,0.093590


**Вывод**

Клиенты с целью покупки автомобиля чаще остальных возвращают кредита в срок, а реже всех клиенты с целью покупки недвижимости.

## Шаг 4. Общий вывод

- Больше всего вероятность возврата кредита в срок если у клиента есть дети, он Не женат / не замужем, его ежемесяный доход средний, а цель получения кредита это покупка автомобиля.
- Меньше всего вероятность возврата кредита в срок если у клиента нет детей, он вдовец / вдова, его ежемесяный доход высокий, а цель получения кредита это покупка недвижимости.